In [ ]:
import os,sys
#folder = '/content/gdrive/My Drive/dataset/case'
folder1 = '/home/rai/다운로드/emotifymusic/classical'

for filename in os.listdir(folder1):
       infilename = os.path.join(folder1,filename)
       if not os.path.isfile(infilename): continue
       oldbase = os.path.splitext(filename)
       newname = infilename.replace('.tmp', '.mp3')
       output = os.rename(infilename, newname)

In [ ]:
import os
import argparse

from pydub import AudioSegment

formats_to_convert = ['.mp3']

for (dirpath, dirnames, filenames) in os.walk("/home/rai/다운로드/emotifymusic/classical"):
    for filename in filenames:
        if filename.endswith(tuple(formats_to_convert)):

            filepath = dirpath + '/' + filename
            (path, file_extension) = os.path.splitext(filepath)
            file_extension_final = file_extension.replace('.', '')
            try:
                track = AudioSegment.from_file(filepath,
                        file_extension_final)
                wav_filename = filename.replace(file_extension_final, 'wav')
                wav_path = dirpath + '/' + wav_filename
                print('CONVERTING: ' + str(filepath))
                file_handle = track.export(wav_path, format='wav')
                os.remove(filepath)
            except:
                print("ERROR CONVERTING " + str(filepath))

In [9]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

#load_path ="/home/rai/다운로드/emotifymusic/classical/mel"
#save_path = f'{load_path.split(".")[0]}.png'

frame_length = 0.032
frame_stride = 0.010

def Mel_S(wav_file):
    # mel-spectrogram
    # sr: sampling radio default - 22050
    y, sr = librosa.load(wav_file, sr=12000)

    # wav_length = len(y)/sr
    input_nfft = int(round(sr*frame_length))    
    input_stride = int(round(sr*frame_stride))

    S = librosa.feature.melspectrogram(y=y, n_mels=96, n_fft=input_nfft, hop_length=input_stride)

    print("Wav length: {}, Mel_S shape:{}".format(len(y)/sr,np.shape(S)))

    #Mel-Spectrogram plot
#    plt.figure(figsize=(10, 4))
#    librosa.display.specshow(librosa.power_to_db(S, ref=np.max), y_axis='mel', sr=sr, hop_length=input_stride, x_axis='time')
#    plt.colorbar(format='%+2.0f dB')
#    plt.title('Mel-Spectrogram')
#    plt.tight_layout()
#    plt.savefig(save_path, bbox_inches=None, pad_inches=0, dpi=100)
#    plt.show()
   
    return S

In [1]:
man_original_data = '/home/rai/다운로드/emotifymusic/classical/1.wav'
#Mel-Spectrogram 
mel_spec = Mel_S(man_original_data)
# Log convert
# log_S = librosa.logamplitude(mel_spec, ref_power=np.max)
mel_spec

NameError: name 'Mel_S' is not defined

In [ ]:
mel_spec = mel_spec[:, 2000:3400]
np.shape(mel_spec)

In [ ]:
np.save('/home/rai/다운로드/emotifymusic/classical/mel/1', mel_spec)

In [10]:
for n in range(1,101):
    file_name = '/home/rai/다운로드/emotifymusic/classical/' + str(n) + '.wav'
    mel_spec = Mel_S(file_name)
    mel_spec = mel_spec[:, 2000:3400]
    np.save('/home/rai/다운로드/emotifymusic/classical/mel/' + str(n), mel_spec)

Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 42.3445, Mel_S shape:(96, 4235)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S shape:(96, 6003)
Wav length: 60.02941666666667, Mel_S s

In [5]:
from argparse import ArgumentParser
import essentia
from essentia.standard import *
import numpy as np

def melspectrogram(audio, 
                   sampleRate=12000, frameSize=512, hopSize=256, 
                   window='hann', zeroPadding=0, center=True,
                   numberBands=96, lowFrequencyBound=0, highFrequencyBound=None,
                   weighting='linear', warpingFormula='slaneyMel', 
                   normalize='unit_tri'):

    if highFrequencyBound is None:
        highFrequencyBound = sampleRate/2
    
    windowing = Windowing(type=window, normalized=False, zeroPadding=zeroPadding)
    spectrum = Spectrum()
    melbands = MelBands(numberBands=numberBands,
                        sampleRate=sampleRate,
                        lowFrequencyBound=lowFrequencyBound, 
                        highFrequencyBound=highFrequencyBound,
                        inputSize=(frameSize+zeroPadding)//2+1,
                        weighting=weighting,
                        normalize=normalize,
                        warpingFormula=warpingFormula,
                        type='power')
    amp2db = UnaryOperator(type='lin2db', scale=2)

    pool = essentia.Pool()
    for frame in FrameGenerator(audio, 
                                frameSize=frameSize, hopSize=hopSize,
                                startFromZero=not center):
        pool.add('mel', amp2db(melbands(spectrum(windowing(frame)))))

    return pool['mel'].T

def load_audio(filename, sampleRate=12000, segment_duration=None):
    audio = MonoLoader(filename=filename, sampleRate=sampleRate)()
  
    if segment_duration:
        segment_duration = round(segment_duration*sampleRate)
        segment_start = (len(audio) - segment_duration) // 2
        segment_end = segment_start + segment_duration
    else:
        segment_start = 0
        segment_end = len(audio)

    if segment_start < 0 or segment_end > len(audio):
        raise ValueError('Segment duration is larger than the input audio duration')

    return audio[segment_start:segment_end]

def analyze(audio_file, npy_file, full_audio):
    if full_audio:
      # Analyze full audio duration.
      segment_duration=None
    else:
      # Duration for the Choi's VGG model.
      segment_duration=29.1

    audio = load_audio(audio_file, segment_duration=segment_duration)
    mel = melspectrogram(audio)
    np.save(npy_file, mel, allow_pickle=False)
    return

In [32]:
for n in range(1,101):
    file_name = '/home/rai/다운로드/emotifymusic/pop/' + str(n) + '.mp3'
    save_name = '/home/rai/다운로드/emotifymusic/pop/mel/' + str(n)
    analyze(file_name, save_name, 1)
    print('%d converted'%n)
#    mel_spec = mel_spec[:, 2000:3400]
#    np.save('/home/rai/다운로드/emotifymusic/classical/mel/' + str(n), mel_spec)

1 converted
2 converted
3 converted
4 converted
5 converted
6 converted
7 converted
8 converted
9 converted
10 converted
11 converted
12 converted
13 converted
14 converted
15 converted
16 converted
17 converted
18 converted
19 converted
20 converted
21 converted
22 converted
23 converted
24 converted
25 converted
26 converted
27 converted
28 converted
29 converted
30 converted
31 converted
32 converted
33 converted
34 converted
35 converted
36 converted
37 converted
38 converted
39 converted
40 converted
41 converted
42 converted
43 converted
44 converted
45 converted
46 converted
47 converted
48 converted
49 converted
50 converted
51 converted
52 converted
53 converted
54 converted
55 converted
56 converted
57 converted
58 converted
59 converted
60 converted
61 converted
62 converted
63 converted
64 converted
65 converted
66 converted
67 converted
68 converted
69 converted
70 converted
71 converted
72 converted
73 converted
74 converted
75 converted
76 converted
77 converted
78 conve

In [27]:
npy = np.load('/home/rai/다운로드/emotifymusic/rock/mel/4.npy')
np.mean(npy)

-21.62681

In [29]:
file_name = '/home/rai/다운로드/mtg-jamendo-dataset-master/path/audio/00/7400.mp3'
save_name = '/home/rai/다운로드/mtg-jamendo-dataset-master/path/audio/00/7400'
analyze(file_name, save_name, 1)